In [322]:
import torch
from torch import nn
import matplotlib.pyplot as plt

from VOC_dataset import VOCDataset
from config import DefaultConfig
from fcos import FCOS
from loss import GenTargets, LOSS

train_dataset=VOCDataset("f:/Dataset/VOC/VOC2007/",resize_size=[512,800],split='train')
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=1,shuffle=True,collate_fn=train_dataset.collate_fn)

INFO=====>voc dataset init finished  ! !


In [265]:
config = DefaultConfig
fcos_body = FCOS(config)
target_layer=GenTargets(strides=config.strides,limit_range=config.limit_range)

In [290]:
for epoch_step, data in enumerate(train_loader):
    batch_imgs, batch_boxes, batch_classes = data
    
    out = fcos_body(batch_imgs)
    targets=target_layer([out,batch_boxes,batch_classes])
    
    if epoch_step == 0:
        break

### GenTargets

In [163]:
strides = config.strides
limit_ranges = config.limit_range

print(strides, limit_ranges)

[8, 16, 32, 64, 128] [[-1, 64], [64, 128], [128, 256], [256, 512], [512, 999999]]


In [259]:
# cls_logits  list contains five [batch_size,class_num,h,w]  
# cnt_logits  list contains five [batch_size,1,h,w]  
# reg_preds   list contains five [batch_size,4,h,w]  
# gt_boxes [batch_size,m,4]  FloatTensor  
# classes [batch_size,m]  LongTensor
cls_logits, cnt_logits, reg_preds = out
gt_boxes = batch_boxes
classes = batch_classes

cls_targets_all_level = []
cnt_targets_all_level = []
reg_targets_all_level = []

level = 0
level_out = [cls_logits[level], cnt_logits[level], reg_preds[level]]
# for level in range(len(cls_logits)):
#     level_out = [cls_logits[level], cnt_logits[level], reg_preds[level]]
#     level_targets = 

In [260]:
# ._gen_level_targets(level_out,gt_boxes,classes,self.strides[level], self.limit_range[level])
stride = strides[level]
limit_range = limit_ranges[level]
sample_radiu_ratio = 1.5

cls_logits, cnt_logits, reg_preds = level_out
batch_size=cls_logits.shape[0]
class_num = cls_logits.shape[1]
m = gt_boxes.shape[1]

cls_logits = cls_logits.permute(0, 2, 3, 1) #[batch_size,h,w,class_num]  
coords=coords_fmap2orig(cls_logits,stride)
cls_logits=cls_logits.reshape((batch_size,-1,class_num)) #[batch_size,h*w,class_num]  

cnt_logits=cnt_logits.permute(0,2,3,1)
cnt_logits=cnt_logits.reshape((batch_size,-1,1))

reg_preds=reg_preds.permute(0,2,3,1)
reg_preds=reg_preds.reshape((batch_size,-1,4))

In [261]:
h_mul_w=cls_logits.shape[1]

x=coords[:,0]
y=coords[:,1]

l_off = x[None, :, None] - gt_boxes[...,0][:, None, :] # x_min
t_off = y[None,:,None] - gt_boxes[...,1][:,None,:] # y_min
r_off = gt_boxes[...,2][:,None,:] - x[None,:,None] # x_max
b_off = gt_boxes[...,3][:,None,:] - y[None,:,None] # y_max

ltrb_off = torch.stack([l_off, t_off, r_off, b_off],dim=-1) # [batch_size,h*w,m,4]
areas = (ltrb_off[...,0] + ltrb_off[...,2]) * (ltrb_off[...,1] + ltrb_off[...,3]) # [batch_size,h*w,m]

off_min = torch.min(ltrb_off, dim=-1)[0] # [batch_size,h*w,m]
off_max = torch.max(ltrb_off,dim=-1)[0] # [batch_size,h*w,m]

mask_in_gtboxes = off_min > 0
mask_in_level = (off_max > limit_range[0])&(off_max<=limit_range[1])

radiu = stride*sample_radiu_ratio
gt_center_x = (gt_boxes[...,0]+gt_boxes[...,2])/2
gt_center_y = (gt_boxes[...,1]+gt_boxes[...,3])/2

c_l_off = x[None,:,None] - gt_center_x[:,None,:]
c_t_off = y[None,:,None] - gt_center_y[:,None,:]
c_r_off = gt_center_x[:,None,:] - x[None,:,None]
c_b_off = gt_center_y[:,None,:] - y[None,:,None]

c_ltrb_off = torch.stack([c_l_off, c_t_off, c_r_off, c_b_off], dim=-1)
c_off_max = torch.max(c_ltrb_off,dim=-1)[0]

mask_center = c_off_max < radiu
mask_pos = mask_in_gtboxes & mask_in_level & mask_center

areas[~mask_pos]=99999999 # make negative
areas_min_ind = torch.min(areas, dim=-1)[1]

reg_targets=ltrb_off[torch.zeros_like(areas,dtype=torch.bool).scatter_(-1,areas_min_ind.unsqueeze(dim=-1),1)]
reg_targets=torch.reshape(reg_targets,(batch_size,-1,4))

classes = torch.broadcast_tensors(classes[:,None,:], areas.long())[0]
cls_targets=classes[torch.zeros_like(areas,dtype=torch.bool).scatter_(-1,areas_min_ind.unsqueeze(dim=-1),1)]
cls_targets=torch.reshape(cls_targets,(batch_size,-1,1))

left_right_min = torch.min(reg_targets[..., 0], reg_targets[..., 2])
left_right_max = torch.max(reg_targets[..., 0], reg_targets[..., 2])
top_bottom_min = torch.min(reg_targets[..., 1], reg_targets[..., 3])
top_bottom_max = torch.max(reg_targets[..., 1], reg_targets[..., 3])

cnt_targets=((left_right_min*top_bottom_min)/(left_right_max*top_bottom_max+1e-10)).sqrt().unsqueeze(dim=-1)

mask_pos_2 = mask_pos.long().sum(dim=-1)
mask_pos_2 = mask_pos_2>=1

cls_targets[~mask_pos_2]=0
cnt_targets[~mask_pos_2]=-1
reg_targets[~mask_pos_2]=-1

In [95]:
# coords=coords_fmap2orig(cls_logits,stride).to(device=gt_boxes.device)#[h*w,2]
h,w=cls_logits.shape[1:3]
stride = strides[level]
shifts_x = torch.arange(0, w*stride, stride, dtype=torch.float32)
shifts_y = torch.arange(0, h*stride, stride, dtype=torch.float32)

shift_y, shift_x = torch.meshgrid(shifts_y, shifts_x)
shift_x = torch.reshape(shift_x, [-1])
shift_y = torch.reshape(shift_y, [-1])

coords = torch.stack([shift_x, shift_y], -1) + stride // 2

In [83]:
def coords_fmap2orig(feature,stride):
    h,w=feature.shape[1:3]
    shifts_x = torch.arange(0, w * stride, stride, dtype=torch.float32)
    shifts_y = torch.arange(0, h * stride, stride, dtype=torch.float32)

    shift_y, shift_x = torch.meshgrid(shifts_y, shifts_x)
    shift_x = torch.reshape(shift_x, [-1])
    shift_y = torch.reshape(shift_y, [-1])
    coords = torch.stack([shift_x, shift_y], -1) + stride // 2
    return coords

### LOSS

In [333]:
cls_logits, cnt_logits, reg_preds = out
cls_targets, cnt_targets, reg_targets = targets

mask_pos = (cnt_targets>-1).squeeze(dim=-1)
cls_loss=compute_cls_loss(cls_logits,cls_targets,mask_pos).mean()
cnt_loss=compute_cnt_loss(cnt_logits,cnt_targets,mask_pos).mean()
reg_loss=compute_reg_loss(reg_preds,reg_targets,mask_pos, mode='iou').mean()

total_loss=cls_loss+cnt_loss+reg_loss

In [334]:
total_loss

tensor(13.0966, grad_fn=<AddBackward0>)

In [317]:
# compute_cls_loss(preds,targets,mask)
batch_size=cls_targets.shape[0]
class_num=cls_logits[0].shape[1]
preds_reshape=[]

mask = mask_pos.unsqueeze(dim=-1)
num_pos = torch.sum(mask,dim=[1,2]).clamp_(min=1).float()

for cls_logit in cls_logits:
    cls_logit = cls_logit.permute(0,2,3,1)
    cls_logit = torch.reshape(cls_logit, [batch_size,-1,class_num])
    preds_reshape.append(cls_logit)
    
cls_preds = torch.cat(preds_reshape,dim=1)

loss=[]
for batch_index in range(batch_size):
    pred_pos=cls_preds[batch_index]#[sum(_h*_w),class_num]
    target_pos=cls_targets[batch_index]#[sum(_h*_w),1]
    target_pos=(torch.arange(1,class_num+1)[None,:]==target_pos).float()#sparse-->onehot
    loss.append(focal_loss_from_logits(pred_pos,target_pos).view(1))

In [327]:
def compute_cls_loss(preds,targets,mask):
    batch_size=targets.shape[0]
    preds_reshape=[]
    class_num=preds[0].shape[1]
    mask=mask.unsqueeze(dim=-1)
    # mask=targets>-1#[batch_size,sum(_h*_w),1]
    num_pos=torch.sum(mask,dim=[1,2]).clamp_(min=1).float()#[batch_size,]
    for pred in preds:
        pred=pred.permute(0,2,3,1)
        pred=torch.reshape(pred,[batch_size,-1,class_num])
        preds_reshape.append(pred)
    preds=torch.cat(preds_reshape,dim=1)#[batch_size,sum(_h*_w),class_num]
    assert preds.shape[:2]==targets.shape[:2]
    loss=[]
    for batch_index in range(batch_size):
        pred_pos=preds[batch_index]#[sum(_h*_w),class_num]
        target_pos=targets[batch_index]#[sum(_h*_w),1]
        target_pos=(torch.arange(1,class_num+1,device=target_pos.device)[None,:]==target_pos).float()#sparse-->onehot
        loss.append(focal_loss_from_logits(pred_pos,target_pos).view(1))
    return torch.cat(loss,dim=0)/num_pos

In [323]:
def compute_reg_loss(preds,targets,mask,mode='giou'):
    batch_size=targets.shape[0]
    c=targets.shape[-1]
    preds_reshape=[]
    # mask=targets>-1#[batch_size,sum(_h*_w),4]
    num_pos=torch.sum(mask,dim=1).clamp_(min=1).float()#[batch_size,]
    for pred in preds:
        pred=pred.permute(0,2,3,1)
        pred=torch.reshape(pred,[batch_size,-1,c])
        preds_reshape.append(pred)
    preds=torch.cat(preds_reshape,dim=1)
    assert preds.shape==targets.shape#[batch_size,sum(_h*_w),4]
    loss=[]
    for batch_index in range(batch_size):
        pred_pos=preds[batch_index][mask[batch_index]]#[num_pos_b,4]
        target_pos=targets[batch_index][mask[batch_index]]#[num_pos_b,4]
        assert len(pred_pos.shape)==2
        if mode=='iou':
            loss.append(iou_loss(pred_pos,target_pos).view(1))
        elif mode=='giou':
            loss.append(giou_loss(pred_pos,target_pos).view(1))
        else:
            raise NotImplementedError("reg loss only implemented ['iou','giou']")
    return torch.cat(loss,dim=0)/num_pos#[batch_size,]

In [324]:
def compute_cnt_loss(preds,targets,mask):
    batch_size=targets.shape[0]
    c=targets.shape[-1]
    preds_reshape=[]
    mask=mask.unsqueeze(dim=-1)
    # mask=targets>-1#[batch_size,sum(_h*_w),1]
    num_pos=torch.sum(mask,dim=[1,2]).clamp_(min=1).float()#[batch_size,]
    for pred in preds:
        pred=pred.permute(0,2,3,1)
        pred=torch.reshape(pred,[batch_size,-1,c])
        preds_reshape.append(pred)
    preds=torch.cat(preds_reshape,dim=1)
    assert preds.shape==targets.shape#[batch_size,sum(_h*_w),1]
    loss=[]
    for batch_index in range(batch_size):
        pred_pos=preds[batch_index][mask[batch_index]]#[num_pos_b,]
        target_pos=targets[batch_index][mask[batch_index]]#[num_pos_b,]
        assert len(pred_pos.shape)==1
        loss.append(nn.functional.binary_cross_entropy_with_logits(input=pred_pos,target=target_pos,reduction='sum').view(1))
    return torch.cat(loss,dim=0)/num_pos#[batch_size,]

In [316]:
def focal_loss_from_logits(preds,targets,gamma=2.0,alpha=0.25):
    preds=preds.sigmoid()
    pt=preds*targets+(1.0-preds)*(1.0-targets)
    w=alpha*(1.0-targets)+(1.0-alpha)*targets
    loss=-w*torch.pow((1.0-pt),gamma)*pt.log()
    return loss.sum()

In [321]:
def iou_loss(preds,targets):
    lt=torch.min(preds[:,:2],targets[:,:2])
    rb=torch.min(preds[:,2:],targets[:,2:])
    wh=(rb+lt).clamp(min=0)
    overlap=wh[:,0]*wh[:,1]#[n]
    area1=(preds[:,2]+preds[:,0])*(preds[:,3]+preds[:,1])
    area2=(targets[:,2]+targets[:,0])*(targets[:,3]+targets[:,1])
    iou=overlap/(area1+area2-overlap)
    loss=-iou.clamp(min=1e-6).log()
    return loss.sum()


In [310]:
batch_index = 0
pred_pos=cls_preds[batch_index]
target_pos=cls_targets[batch_index]
target_pos=(torch.arange(1,class_num+1)[None,:]==target_pos).float()

In [315]:
target_pos

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [314]:
target_pos.shape

torch.Size([7983, 80])